In [1]:
import os
from os import environ
from selenium import webdriver
from selenium.webdriver.support.select import Select
import dateparser
import datetime
import time
import random
import requests

driver = webdriver.Chrome()

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
API_KEY_2CAPTCHA = os.getenv("API_KEY_2CAPTCHA")

In [3]:
CALIFORNIA_ID = 'taskCID'
REAL_ID = 'taskRID'

# edit these:
task_types = [CALIFORNIA_ID, REAL_ID] 
name = 'First'
last_name = 'Last'
phone_number = '6280000000'

In [4]:
driver.get("https://www.dmv.ca.gov/wasapp/foa/clear.do?goTo=officeVisit&localeName=en")

In [27]:
task_selector_id_by_count = ['one_task', 'two_tasks', 'three_tasks']
task_selector_id = task_selector_id_by_count[len(task_types)-1]
task_selector = driver.find_element_by_id(task_selector_id)
task_selector.click()

In [28]:
for task_type in task_types:
    task_type_selector = driver.find_element_by_id(task_type)
    if not task_type_selector.is_selected():
        task_type_selector.click() 

In [29]:
name_field = driver.find_element_by_name('firstName')
name_field.clear()
name_field.send_keys(name)
lastname_field = driver.find_element_by_name('lastName')
lastname_field.clear()
lastname_field.send_keys(last_name)

In [30]:
phone_area_field = driver.find_element_by_name('telArea')
phone_area_field.send_keys(phone_number)

In [31]:
results = {}
"""
<button class="rc-button-default goog-inline-block" title="" value="" id="recaptcha-verify-button" tabindex="0">Verify</button>""";

In [32]:
target_office_names = [
                'SAN FRANCISCO',
                'DALY CITY',
                'OAKLAND CLAREMONT',
                'OAKLAND COLISEUM',
                'HAYWARD',
                'FREMONT',
                'EL CERRITO',
                'CONCORD',
                'WALNUT CREEK',
                'PITTSBURG',
                'PLEASANTON']

In [33]:
def uncaptcha():
    dos_captchas = driver.find_elements_by_class_name('rc-doscaptcha-body-text')
    # TODO: this is happening in iframe, detect
    if (len(dos_captchas) > 0):
        print('Dos banner - throttling')
        throttle()
        raise ValueError("Boom")
        return
    
    recaptcha_response_fields = driver.find_elements_by_class_name('g-recaptcha-response')
    if (len(recaptcha_response_fields) == 0):
        print('No captcha it seems - skipping')
        return

    s = requests.Session()
    recaptcha_input_field = driver.find_element_by_class_name('g-recaptcha')
    googlekey = recaptcha_input_field.get_attribute('data-sitekey')
    pageurl = driver.current_url
    api_key = API_KEY_2CAPTCHA
    method = 'userrecaptcha'
    captcha_submission_response = s.post(f"https://2captcha.com/in.php?key={api_key}" +
                        f"&method={method}&googlekey={googlekey}&pageurl={pageurl}&json=1")
    request_id = captcha_submission_response.json()['request']
    uncap = None
    for i in range(10):
        print(f'Waiting request #{request_id} {i}')
        time.sleep(10)
        # {'status': 0, 'request': 'CAPCHA_NOT_READY'}
        # {'status': 1, 'request': ''}
        try:
            recaptcha_answer = s.get(f"https://2captcha.com/res.php?key={api_key}&action=get&id={request_id}&json=1")
            if (recaptcha_answer.json()['status'] == 1):
                uncap = recaptcha_answer.json()['request']
                break
        except Exception as e:
            print(e)

    recaptcha_response_field = driver.find_element_by_class_name('g-recaptcha-response')
    driver.execute_script("arguments[0].value='"+uncap+"';", recaptcha_response_field);
    driver.find_element_by_id("findOffice").submit();    

In [34]:
def throttle():
    # TODO: progressbar here would be nice
    sleep_seconds = random.randint(40,140)
    print(f"Sleeping {sleep_seconds}")
    time.sleep(sleep_seconds)

In [35]:
# before: https://www.dmv.ca.gov/wasapp/foa/officeVisit.do
selector_field = driver.find_element_by_name('officeId')
selector = Select(selector_field)
office_names = selector_field.text.split('\n')

In [36]:
office_names_idx = {office_name: idx for idx, office_name in enumerate(office_names)}

In [37]:
#office_names.insert(0, '<empty>')
offices_count = len(selector.options)
for office_name in target_office_names:
    idx = office_names_idx[office_name]
    print(str(idx) + ": " + office_name)
    if (not office_name in target_office_names):
        continue

    try:
        selector_field = driver.find_element_by_name('officeId')
        Select(selector_field).select_by_index(idx)
        submit = driver.find_element_by_class_name('btn-primary')
        submit.click()
        uncaptcha()

        table_rows = driver.find_elements_by_css_selector('tbody > tr > td > p.no-margin-bottom')
        address = 'not available'
        
        if len(table_rows)>0:
            address = table_rows[0].text
        if ('not available' in address) or ('no appointment is available' in address) or ('all appointments at this office are currently taken' in address):
            print("Skipping {idx}({name}): {address}".format(idx=idx, name=office_name, address=address))
        else:
            datetime_str = table_rows[2].text
            datetime_val = dateparser.parse(datetime_str)
            results[address] = datetime_val

        driver.back()
        throttle()
        # TODO: report good
    except:
        print("Problem processing element " + str(idx))
        raise

85: OAKLAND CLAREMONT
Waiting request #62610749194 0
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Waiting request #62610749194 1
Skipping 85(OAKLAND CLAREMONT): not available
Sleeping 90
86: OAKLAND COLISEUM
Waiting request #62610755303 0
Waiting request #62610755303 1
Waiting request #62610755303 2
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Waiting request #62610755303 3
Skipping 86(OAKLAND COLISEUM): not available
Sleeping 82
40: FREMONT
Waiting request #62610762892 0
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Waiting request #62610762892 1
Skipping 40(FREMONT): not available
Sleeping 87
27: DALY CITY
Waiting request #62610769696 0
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Waiting request #62610769696 1
Skipping 27(DALY CITY): not available
Sleeping 105
53: HAYWARD
Problem processing elem

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="officeId"]"}
  (Session info: chrome=78.0.3904.97)


In [ ]:
selector_field = driver.find_element_by_name('officeId')
Select(selector_field).select_by_index(1)

In [ ]:
sorted([date.isoformat() + ' ' + address for address, date in results.items()])

In [ ]:
driver.close()

In [ ]:
driver.find_elements_by_class_name('g-recaptcha-response')